# Fake News Prediction

### Importing the Dependencies

In [78]:
import numpy as np 
import pandas as pd 
import re
from nltk.corpus import stopwords  #Natural Language Tool Kit
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Data Preprocessing

In [12]:
news_data = pd.read_csv('train.csv')
data = news_data.copy()

In [15]:
data.shape

(20800, 5)

In [16]:
data.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [17]:
# Counting the missing values
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [18]:
# Replacing the null values with empty string
data = data.fillna(' ')

In [67]:
# Merging the title and author name 
data['Content'] = data['title']+" "+data['author']

In [68]:
print(data['Content'])

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Consortiumne...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799              What Keeps the F-35 Alive David Swanson
Name: Content, Length: 20800, dtype: object


### Stemming 
Stemming is the process of reducing a word to its Root word

In [70]:
port_stem = PorterStemmer()

In [71]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) # Excludes all characters except a-z A-Z and (',','.'..) replaced by ' '
    #stemmed_content = re.sub('[^a-z]',' ',content)
    stemmed_content = stemmed_content.lower() # converts all words to lower case
    stemmed_content = stemmed_content.split() # converst to a list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content


In [72]:
data['Content'] = data['Content'].apply(stemming)

In [73]:
data['Content']

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get fire consortiumnew com
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkan alex ...
20799                            keep f aliv david swanson
Name: Content, Length: 20800, dtype: object

In [74]:
X = data['Content'].values
Y = data['label'].values

In [80]:
# Converting textual data to numerical data
vectorizer = TfidfVectorizer() 
vectorizer.fit(X)

X = vectorizer.transform(X)

In [81]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210687 stored elements and shape (20800, 17128)>
  Coords	Values
  (0, 267)	0.2701012497770876
  (0, 2483)	0.36765196867972083
  (0, 2959)	0.24684501285337127
  (0, 3600)	0.3598939188262558
  (0, 3792)	0.27053324808454915
  (0, 4973)	0.23331696690935097
  (0, 7005)	0.2187416908935914
  (0, 7692)	0.24785219520671598
  (0, 8630)	0.2921251408704368
  (0, 8909)	0.36359638063260746
  (0, 13473)	0.2565896679337956
  (0, 15686)	0.2848506356272864
  (1, 1497)	0.2939891562094648
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 2813)	0.19094574062359204
  (1, 3568)	0.26373768806048464
  (1, 5503)	0.7143299355715573
  (1, 6816)	0.1904660198296849
  (1, 16799)	0.30071745655510157
  (2, 2943)	0.3179886800654691
  (2, 3103)	0.46097489583229645
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  :	:
  (20797, 3643)	0.2115550061362374
  (20797, 7042)	0.21799048897828685
  (2079

### Splitting the train test data

In [82]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

### Training the Logistic Regression Model

In [83]:
model = LogisticRegression()

In [84]:
model.fit(X_train,Y_train)

LogisticRegression()

### Evaluation

In [85]:
# Accuracy of the training data
X_train_prediction = model.predict(X_train)
train_data_accuracy = accuracy_score(X_train_prediction,Y_train)
print("Accuracy score:",train_data_accuracy)

Accuracy score: 0.9863581730769231


In [86]:
# Accuracy of the test data
X_test_prediction = model.predict(X_test)
train_data_accuracy = accuracy_score(X_test_prediction,Y_test)
print("Accuracy score:",train_data_accuracy)

Accuracy score: 0.9790865384615385


### Building a Predictive System

In [92]:
# Taking a random column from the test dataset 
x_new = X_test[22]
prediction = model.predict(x_new)

print(prediction)
if prediction[0]==0:
    print("The news is real")
else:
    print("The news is fake")

[0]
The news is real


In [93]:
# Checking the original label in the Y_test
print(Y_test[22])

0
